In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('figure',figsize=(20,12))
rc('font',size=12)

from scipy.signal import medfilt
from scipy.interpolate import interp1d
import glob
import scipy.ndimage.filters as scfilt

from qubicpack.qubicfp import qubicfp
from qubic import fibtools as ft

from importlib import reload
import healpy as hp
import iminuit
from iminuit.cost import LeastSquares
from sklearn.cluster import DBSCAN


import time_domain_tools as tdt


In [ ]:
#### Find files with reconstructed time constants for data taken on 2019-11-12
# Each file has the following fields in a np.savetxt format:
# ch2vals, ndfvals, risefit, riseerr, fallfit, 
# fallerr, t0fit, t0err, new_ch2vals, new_ndfvals, new_risefit, 
# new_riseerr, new_fallfit, new_fallerr, ok, validfit
files = np.array(glob.glob('Tcst-Fit-JC_2019-11-12*.txt'))

### Sort them
allbias2019 = np.zeros(len(files))
for i in range(len(files)):
    allbias2019[i] = float(files[i].split('_')[-1][:-4])

s = argsort(allbias2019)
allbias2019 = allbias2019[s]
files = files[s]
alldata2019 = np.zeros((16, len(files), 256))

for i in range(len(files)):
    print(files[i], allbias2019[i])
    alldata2019[:,i,:] = np.loadtxt(files[i])
ch2vals2019, ndfvals2019, risefit2019, riseerr2019, fallfit2019, fallerr2019, t0fit2019, t0err2019, new_ch2vals2019, new_ndfvals2019, new_risefit2019, new_riseerr2019, new_fallfit2019, new_fallerr2019, ok2019, validfit2019 = alldata2019
ok2019 = ok2019.astype(bool)

In [ ]:
#### Find files with reconstructed time constants for data taken on 2022-08-24
# Each file has the following fields in a np.savetxt format:
# ch2vals, ndfvals, risefit, riseerr, fallfit, 
# fallerr, t0fit, t0err, new_ch2vals, new_ndfvals, new_risefit, 
# new_riseerr, new_fallfit, new_fallerr, ok, validfit
files = np.array(glob.glob('Tcst-Fit-JC_2022-08-24*.txt'))

### Sort them
allbias = np.zeros(len(files))
for i in range(len(files)):
    allbias[i] = float(files[i].split('_')[-1][:-4])

s = argsort(allbias)
allbias = allbias[s]
files = files[s]
alldata = np.zeros((16, len(files), 256))

for i in range(len(files)):
    print(files[i])
    alldata[:,i,:] = np.loadtxt(files[i])
ch2vals, ndfvals, risefit, riseerr, fallfit, fallerr, t0fit, t0err, new_ch2vals, new_ndfvals, new_risefit, new_riseerr, new_fallfit, new_fallerr, ok, validfit = alldata
ok = ok.astype(bool)

In [ ]:
TESNums = np.arange(256)+1
alwaysok = np.product(ok, axis=0).astype(bool)
nbfullok = np.sum(alwaysok)
print(nbfullok)

subplot(2,2,1)
imshow(ok, extent=[-0.5,255.5,np.min(allbias), np.max(allbias)], interpolation='None', aspect='auto')
xlabel('TES#')
ylabel('Bias [V]')
title('Always valid: {}'.format(nbfullok))
for i in range(nbfullok):
    axvline(x=TESNums[alwaysok][i]-1, color='r')

subplot(2,2,2)
plot(allbias, np.sum(ok,axis=1), lw=3)
xlabel('Bias [V]')
ylabel('Number of good TES')
    
numsok = TESNums[alwaysok]
print(numsok)

mrisegood = np.zeros(len(allbias))
srisegood = np.zeros(len(allbias))
mfallgood = np.zeros(len(allbias))
sfallgood = np.zeros(len(allbias))
for i in range(len(allbias)):
    mrisegood[i], srisegood[i] = ft.meancut(new_risefit[i, alwaysok], 3)
    mfallgood[i], sfallgood[i] = ft.meancut(new_fallfit[i, alwaysok], 3)



subplot(2,2,3)
for i in range(len(numsok)):
    errorbar(allbias, new_risefit[:,numsok[i]-1], yerr=new_riseerr[:,numsok[i]-1], fmt='o-', label='TES#{}'.format(TESNums[numsok[i]-1]), alpha=0.5)
plot(allbias, mrisegood, 'k', lw=4)
plot(allbias, mrisegood-srisegood, 'k', lw=2)
plot(allbias, mrisegood+srisegood, 'k', lw=2)
pp = polyfit(allbias, mrisegood, 1, w=1./srisegood*2)
p = np.poly1d(pp)
plot(allbias, p(allbias), lw=3, color='r', label='{0:3.2f}x+{1:3.2f}'.format(pp[0], pp[1]))
legend(fontsize=6, loc='upper right')
xlabel('Bias [V]')
ylabel('Rise Time [s]')
ylim(0,1)
title('Always good TESs')

subplot(2,2,4)
for i in range(len(numsok)):
    errorbar(allbias, new_fallfit[:,numsok[i]-1], yerr=new_fallerr[:,numsok[i]-1], fmt='o-', label='TES#{}'.format(TESNums[numsok[i]-1]), alpha=0.5)
plot(allbias, mfallgood, 'k', lw=4)
plot(allbias, mfallgood-sfallgood, 'k', lw=2)
plot(allbias, mfallgood+sfallgood, 'k', lw=2)
pp = polyfit(allbias, mfallgood, 1, w=1./sfallgood*2)
p = np.poly1d(pp)
plot(allbias, p(allbias), lw=3, color='r', label='{0:3.2f}x+{1:3.2f}'.format(pp[0], pp[1]))
legend(fontsize=6, loc='upper right')
xlabel('Bias [V]')
ylabel('Fall Time [s]')
ylim(0,1)
title('Always good TESs')

In [ ]:
def plot_contours_pol(pp, covpp, xvalues, nbmc=1000, add_label='', nsig=1, **kwargs):
    allvals = np.random.multivariate_normal(pp, covpp, size=nbmc)
    allyvals = np.zeros((nbmc, len(xvalues)))
    for i in range(nbmc):
        allyvals[i,:] = np.poly1d(allvals[i,:])(xvalues)
    ss = np.std(allyvals, axis=0)
    fill_between(xvalues, np.poly1d(pp)(xvalues)-ss, y2=np.poly1d(pp)(xvalues)+ss, alpha=0.3, **kwargs)
    plot(xvalues, np.poly1d(pp)(xvalues), **kwargs, label=add_label+'{0:5.2g}x +/- {1:5.2g}'.format(pp[0], pp[1]))

    

In [ ]:
mrisegood = np.zeros(len(allbias))
srisegood = np.zeros(len(allbias))
mfallgood = np.zeros(len(allbias))
sfallgood = np.zeros(len(allbias))
for i in range(len(allbias)):
    mrisegood[i], srisegood[i] = ft.meancut(new_risefit[i, ok[i,:]], 2)
    mfallgood[i], sfallgood[i] = ft.meancut(new_fallfit[i, ok[i,:]], 2)

mrisegood2019 = np.zeros(len(allbias2019))
srisegood2019 = np.zeros(len(allbias2019))
mfallgood2019 = np.zeros(len(allbias2019))
sfallgood2019 = np.zeros(len(allbias2019))
for i in range(len(allbias2019)):
    mrisegood2019[i], srisegood2019[i] = ft.meancut(new_risefit2019[i, ok2019[i,:]], 2)
    mfallgood2019[i], sfallgood2019[i] = ft.meancut(new_fallfit2019[i, ok2019[i,:]], 2)

subplot(2,2,1)
errorbar(allbias, mrisegood, yerr=srisegood, fmt='ko', label='Data 2022')
pprise, covpprise = polyfit(allbias, mrisegood, 1, w=1./srisegood**2, cov=True)
plot_contours_pol(pprise, covpprise, allbias, color='k', add_label='Data 2022: ')

errorbar(allbias2019, mrisegood2019, yerr=srisegood2019, fmt='bo', alpha=0.3, label='Data 2019')
pprise2019, covpprise2019 = polyfit(allbias2019, mrisegood2019, 1, w=1./srisegood2019**2, cov=True)
plot_contours_pol(pprise2019, covpprise2019, allbias2019, color='b', add_label='Data 2019: ')



xlabel('Bias [V]')
ylabel('Rise Time [s]')
title('Rise-Time: good TESs')
ylim(0,0.5)
legend()

subplot(2,2,2)
errorbar(allbias, mfallgood, yerr=sfallgood, fmt='ko', label='Data 2022')
ppfall, covppfall = polyfit(allbias, mfallgood, 1, w=1./sfallgood**2, cov=True)
plot_contours_pol(ppfall, covppfall, allbias, color='k', add_label='Data 2022: ')

errorbar(allbias2019, mfallgood2019, yerr=sfallgood2019, fmt='bo', alpha=0.3, label='Data 2019')
ppfall2019, covppfall2019 = polyfit(allbias2019, mfallgood2019, 1, w=1./sfallgood2019**2, cov=True)
plot_contours_pol(ppfall2019, covppfall2019, allbias2019, color='b', add_label='Data 2019: ')


xlabel('Bias [V]')
ylabel('Fall Time [s]')
title('Fall-Time: good TESs')
ylim(0,0.5)
legend()

# Now let's look for each TES

In [ ]:
### We need the things below to be able to plot with the focal plane layout

from qubicpack.utilities import Qubic_DataDir
import qubic
from qubic import DataHandling as DH
from qubic import progress_bar
from qubic import selfcal_lib as scal

basedir = Qubic_DataDir()
print('basedir : ', basedir)
dictfilename = basedir + '/dicts/global_source_oneDet.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
q = qubic.QubicInstrument(d)



In [ ]:
rc('font',size=6)
fig=figure(figsize=(30, 30))
fig.subplots_adjust(hspace=0,wspace=0)
bar=progress_bar(256, 'Display focal plane')
x=np.linspace(-0.0504, -0.0024, 17)
y=np.linspace(-0.0024, -0.0504, 17)
X, Y = np.meshgrid(x, y)
allTES=np.arange(1, 129, 1)
good_tes=np.delete(allTES, np.array([4,36,68,100])-1, axis=0)
k=0
for j in [1, 2]:
    for i in good_tes:
        xtes, ytes, FP_index, index_q= scal.TES_Instru2coord(TES=i, ASIC=j, q=q, frame='ONAFP', verbose=False)
        ind=np.where((np.round(xtes, 4) == np.round(X, 4)) & (np.round(ytes, 4) == np.round(Y, 4)))
        place_graph=ind[0][0]*17+ind[1][0]+1
        mytes=i
        if j == 2:
            mytes+=128

        #subplot(17, 17, place_graph)
        ax = fig.add_subplot(17, 17, place_graph)
        ax.patch.set_facecolor('white')
        ax.patch.set_alpha(1)
        ax.label_outer()

        
        ############ Plotting part here ################
        kok = ok2019[:,i]
        errorbar(allbias2019[kok], new_risefit2019[kok,i], yerr=new_riseerr2019[kok,i], fmt='ko', alpha=0.5)
        plot(allbias2019, poly1d(pprise2019)(allbias2019),'k', lw=3)
        ylim(0, 0.3)
        if kok.sum() >= 3:
            pprise_this, covpprise_this = polyfit(allbias2019[kok], new_risefit2019[kok,i], 1, w=1./new_riseerr2019[kok,i]**2, cov=True)
            plot(allbias2019, poly1d(pprise_this)(allbias2019),'k--')

        kok = ok[:,i]
        errorbar(allbias[kok], new_risefit[kok,i], yerr=new_riseerr[kok,i], fmt='bo', alpha=0.5)
        plot(allbias, poly1d(pprise)(allbias),'b', lw=3)
        ylim(0, 0.3)
        if kok.sum() >= 3:
            pprise_this, covpprise_this = polyfit(allbias[kok], new_risefit[kok,i], 1, w=1./new_riseerr[kok,i]**2, cov=True)
            plot(allbias, poly1d(pprise_this)(allbias),'b--', label='True')
        ################################################
        

        annotate('{} Rise'.format(mytes), xy=(0, 0),  xycoords='axes fraction', fontsize=8, color='black',
                 fontstyle='italic', fontweight='bold', xytext=(0.07,0.85),backgroundcolor='w')
        bar.update()
        k+=1

#savefig(savepdf, format="pdf", bbox_inches="tight")
